### Import packages

In [29]:
import pandas as pd
from datetime import date
import matplotlib.pyplot as plt
import seaborn as sns

### Set-up

For this expercise, we will utilize two files (listed below) from the 'Brazilian E-Commerce Public Dataset' originally posted on [Kaggle]( https://www.kaggle.com/olistbr/brazilian-ecommerce).

In [30]:
# full path of the 'orders' dataset
orders_file = 'https://bitbucket.org/vishal_derive/vcu-data-mining/raw/11253d9f443241b3ce5949802966a80de73af1db/data/olist_orders_dataset.csv'

# full path of the 'customer' dataset
cust_file = 'https://bitbucket.org/vishal_derive/vcu-data-mining/raw/11253d9f443241b3ce5949802966a80de73af1db/data/olist_customers_dataset.csv'

payments_file = 'https://bitbucket.org/vishal_derive/vcu-data-mining/raw/aedab41b6b30a19db9c72e5b82755a118f847d87/data/olist_order_payments_dataset.csv'


### Read data

In [31]:
def read_olist_data(file1, file2, file3, verbose):
    
    # read the data
    orders = pd.read_csv(file1)
    cust = pd.read_csv(file2)
    payments = pd.read_csv(file3)

    # drop unnecessary columns
    #drop_vars = ['order_approved_at', 'order_delivered_carrier_date', 
    #           'order_delivered_customer_date', 'order_estimated_delivery_date']

    
    # date-time conversion
    orders['order_purchase_timestamp'] = pd.to_datetime(orders['order_purchase_timestamp'])
    orders['order_delivered_customer_timestamp'] = pd.to_datetime(orders['order_delivered_customer_date'])
    orders['order_estimated_delivery_timestamp'] = pd.to_datetime(orders['order_estimated_delivery_date'])

    # let's convert the order purchase timestamps into dates
    orders['order_purchase_date'] = orders['order_purchase_timestamp'].dt.date
    orders['order_delivered_customer_formdate'] = orders['order_delivered_customer_timestamp'].dt.date
    orders['order_estimated_delivery_formdate'] = orders['order_estimated_delivery_timestamp'].dt.date

     # extract day of week from the order date
    orders['order_dow'] = orders['order_purchase_timestamp'].dt.weekday_name
    
    # extract month from the order date
    orders['order_month'] = orders['order_purchase_timestamp'].dt.month

    
    # merge orders and cust dataframes
    orders_cust = pd.merge(orders, cust, on='customer_id', how='inner')
    orders_out = pd.merge(orders_cust, payments, on='order_id', how='inner')
    
    # apply filters to (a) discard (incomplete) data after 2018-8-22; see 06_pandas_wrangle.ipynb for the rationale
    #  and (b) keep 'delivered' orders only
    #  we do this here by using a boolean (True/False) mask
    mask = (orders_out['order_purchase_date'] <= date(2018, 8, 22)) & (orders_out['order_status'] == 'delivered')

    orders_out = orders_out[mask]
    
    # discard 'order_status' as we don't need it any more
    orders_out = orders_out.drop('order_status', axis=1)
    
    # let's drop hose columns that we need (for this exercise)
    keep_cols = ['customer_unique_id', 'customer_id','order_id','order_purchase_timestamp', 'order_delivered_customer_timestamp',
                 'order_estimated_delivery_timestamp', 'order_purchase_date', 'order_delivered_customer_formdate', 'order_estimated_delivery_formdate',
                 'order_dow', 'order_month', 'payment_installments', 'payment_type', 'payment_value']

    orders_out = orders_out[keep_cols].sort_values(['customer_unique_id', 'order_purchase_timestamp'])

    #if verbose:
       # print (f'{len(orders_out):,} records in the output  file.')
    
    return orders_out

In [32]:
orders = read_olist_data(orders_file, cust_file, payments_file, 1)

In [33]:
payments = pd.read_csv(payments_file)
#payments[payments['order_id']=='31bc09fdbd701a7a4f9b55b5955b8687']

## Binary Classification model

Let's build a model to predict whether a customer will make a purchase within the next year.

Our predictin window is July 2017 through end of June 2018.

In [34]:
snapshot_date = date(2017, 6, 30)

In [35]:
#filter out only data before snapshot date
mask = orders['order_purchase_timestamp'].dt.date <= snapshot_date

df_raw = orders[mask]

len(orders), len(df_raw)

(100247, 15077)

### Create Attributes 

### Attribute #1: Order recency

In [36]:


# check the max date

df_raw.order_purchase_timestamp.max()

# grab the max purchase date for each customer

cust_recency = df_raw.groupby('customer_unique_id')['order_purchase_timestamp'].max().reset_index()

# count the number of days between the most recent order date and the snapshot date

delta = snapshot_date - cust_recency['order_purchase_timestamp'].dt.date

# grab the number of days (as an integer) from the calculated deltas 

cust_recency['order_recency'] = delta.dt.days

# drop the date (we don't need it any more for this exercise)

cust_recency = cust_recency.drop('order_purchase_timestamp', axis=1)


# how many records (unique customers) do we have for this timeframe?



13855

### Attribute 2: Days defore/after delivery

In [37]:
# count the number of days between the most recent order date and the snapshot date

df_raw['delays'] = (df_raw['order_estimated_delivery_timestamp'].dt.date - df_raw['order_delivered_customer_timestamp'].dt.date).dt.days



/Users/liz/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [39]:
order_delay = df_raw.groupby(['customer_unique_id'])['delays'].sum().reset_index()

13855

### Attribute 3: Orders per Customer

In [41]:
# Count total records (i.e., orders) per customer

cust_orders = df_raw.groupby('customer_unique_id').size().reset_index().rename(columns={0: 'total_orders'})

13855

### ### Dummy attributes 4, 5 and 6: Payment Type, Day of Week, Month

In [42]:
# get dummies

df_raw = pd.get_dummies(df_raw, columns=['payment_type','order_dow','order_month'])

# grab all binary columns that we just created and aggregate at the customer level

#payment type
dumpay = [x for x in df_raw.columns if 'payment_type_' in x]
payment_type = df_raw.groupby('customer_unique_id')[dumpay].sum().reset_index()

# day of week
dumdow = [x for x in df_raw.columns if 'order_dow_' in x]
cust_dow = df_raw.groupby('customer_unique_id')[dumdow].sum().reset_index()

# month
dummonth = [x for x in df_raw.columns if 'order_month_' in x]
cust_month = df_raw.groupby('customer_unique_id')[dummonth].sum().reset_index()


(13855, 13855, 13855)

### Attribute 7: Purchase Dollar Volume

In [44]:
df_raw['order_total']=df_raw['payment_installments']*df_raw['payment_value']

dollar_volume = df_raw.groupby(['customer_unique_id'])['order_total'].sum().reset_index().sort_values('order_total',ascending=False)


13855

### Combine all attributes into a single dataframe

In [267]:
# double check the number of records in each dataframe first

cust_recency.shape, cust_orders.shape, cust_dow.shape, order_delay.shape, cust_month.shape, dollar_volume.shape, payment_type.shape

((13855, 2),
 (13855, 2),
 (13855, 8),
 (13855, 2),
 (13855, 9),
 (13855, 2),
 (13855, 5))

In [47]:
# combine (merge) `cust_recency` with `cust_orders` so that we have both attributes in one dataset

df = pd.merge(cust_recency, cust_orders, on='customer_unique_id') \
     .merge(cust_dow, on='customer_unique_id') \
     .merge(order_delay, on='customer_unique_id') \
     .merge(cust_month, on='customer_unique_id') \
     .merge(dollar_volume, on='customer_unique_id') \
     .merge(payment_type, on='customer_unique_id') 

df.shape

(13855, 24)

### Assign labels (aka the target variable or the dependent variable)

For this exercise, we will assume that the objective of the model is to predict whether a customer will make *at least one purchase* in the future (i.e., within the target window of the model).

**Step 1:** Isolate all orders that were placed within the prediction window.

In [48]:

# select orders that were placed between July 2017 and 2018

mask = (orders['order_purchase_timestamp'].dt.date > snapshot_date) & (orders['order_purchase_timestamp'].dt.date < date(2018,7,1))
target_events_raw = orders[mask]

len(target_events_raw)


72774

In [49]:
# confirm the min and max dates are within the month of August 2018

print (target_events_raw['order_purchase_timestamp'].min(), target_events_raw['order_purchase_timestamp'].max())

2017-07-01 00:04:15 2018-06-30 23:59:49


**Step 2:** Summarize data to get one record per customer.

In [50]:
# count the number of orders (we will convert this into a binary flag later)

target_events = target_events_raw.groupby('customer_unique_id').size().reset_index().rename(columns={0: 'purch'})

target_events.head()

,customer_unique_id,purch
0,0000366f3b9a7992bf8c76cfdf3221e2,1
1,0000b849f77a49e4a4ce2b2a4ca5be3f,1
2,0000f6ccb0745a6a4b88665a16c9f078,1
3,0004aac84e0df4da2b147fca70cf8255,1
4,0004bd2a26a76fe21f786e4fbd80607f,1


In [51]:
print(f'Number of customers who made at least one purchase durnig the prediction window: {len(target_events):,}')

Number of customers who made at least one purchase durnig the prediction window: 67,776


**Step 3:** Merge this dataframe with the `attr` dataframe to create the modeling dataset.

In [52]:
# merge `target_events` with the dataframe that contains customer attributes

df = pd.merge(df, target_events, how='left', on='customer_unique_id')

df.shape

(13855, 25)

In [54]:
# create the binary target flag

df['purch'] = [1 if x > 0 else 0 for x in df['purch']]

df.purch.value_counts()

0    13488
1      367
Name: purch, dtype: int64

In [55]:
# % distribution of the target flag

df.purch.value_counts() / len(df)

0    0.973511
1    0.026489
Name: purch, dtype: float64

Increasing the prediction window helped some, but the target event (1 for purch) is still rare.

In [56]:
# check the correlations

df.corr()['purch']

order_recency              -0.026816
total_orders                0.005163
order_dow_Friday           -0.006783
order_dow_Monday           -0.011389
order_dow_Saturday          0.006374
order_dow_Sunday           -0.000218
order_dow_Thursday          0.011081
order_dow_Tuesday          -0.004688
order_dow_Wednesday         0.013004
delays                      0.016203
order_month_1               0.002737
order_month_2              -0.021879
order_month_3              -0.010577
order_month_4              -0.007809
order_month_5               0.012584
order_month_6               0.021171
order_month_10              0.004326
order_month_12             -0.001401
order_total                 0.009565
payment_type_boleto        -0.020327
payment_type_credit_card    0.027989
payment_type_debit_card    -0.002468
payment_type_voucher       -0.001802
purch                       1.000000
Name: purch, dtype: float64

### Model building

In [ ]:
df.columns


In [57]:
# set-up

preds = df.columns[1:-1]

X = df[preds]
y = df['purch']

### Two-fold partition

In [109]:
# split the dataframe into train(50%) and test(50%)

#from sklearn.model_selection import train_test_split

#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=314)

#len(X_train), len(X_test)

(6927, 6928)

In [176]:
# Shuffle the dataset  
shuffle_df = df.sample(frac=1)

# Define a size for your train set 
train_size = int(0.5* len(df))

# Split your dataset 
train_set = shuffle_df[:train_size]
test_set = shuffle_df[train_size:]

In [177]:
#split the data without using sklearn function. Not recommended since 
#we cannot specify a random seed, and thus cannot duplicate exact results.
X_train=train_set[['order_recency', 'total_orders',
       'order_dow_Friday', 'order_dow_Monday', 'order_dow_Saturday',
       'order_dow_Sunday', 'order_dow_Thursday', 'order_dow_Tuesday',
       'order_dow_Wednesday']]
y_train = train_set['purch']

X_test=test_set[['order_recency', 'total_orders',
       'order_dow_Friday', 'order_dow_Monday', 'order_dow_Saturday',
       'order_dow_Sunday', 'order_dow_Thursday', 'order_dow_Tuesday',
       'order_dow_Wednesday']]
y_test = test_set['purch']

### Run Linear Model

In [178]:
from sklearn.linear_model import LogisticRegression

# define the model object ('liblinear' is recommended for small datasets)
clf = LogisticRegression(solver='liblinear', random_state=314)

# train (fit) the model using the training sample
clf.fit(X_train, y_train)

# make predictions on the test sample
y_preds = clf.predict(X_test)

### Run Decision Tree Model

In [179]:
from sklearn.tree import DecisionTreeClassifier

from sklearn import metrics

from sklearn.metrics import roc_auc_score

In [265]:
# Create Decision Tree classifer object
dtree = DecisionTreeClassifier(max_depth=4, min_samples_split = 0.05, min_samples_leaf = 0.02, random_state=314)

# Train Decision Tree Classifer
dtree = dtree.fit(X_train,y_train)

#Predict the response for test dataset
y_pred_tree = dtree.predict(X_test)

print("The parameters added to the decision tree model to reduce overfitting improved the model's performance")

The parameters added to the decision tree model to reduce overfitting improved the model's performance


### Compare AUC

In [264]:
from sklearn.metrics import roc_auc_score

# calculate the probabilities on the test sample
y_scores = clf.predict_proba(X_test)[:, 1]

# calcualte AUC
log_auc=roc_auc_score(y_test, y_scores)

# calculate the probabilities on the test sample
y_scores_tree = dtree.predict_proba(X_test)[:, 1]

# calcualte AUC
dtree_auc = roc_auc_score(y_test, y_scores_tree)

print("AUC of Logistic Regression:",log_auc),
print("AUC of Decision Tree:",dtree_auc)


AUC of Logistic Regression: 0.5351371175531571
AUC of Decision Tree: 0.5530515760775223


### Compare Confusion Matrices

In [184]:
#Confusion Matrix for Logistic Regression
confusion_log = pd.crosstab(y_test, y_preds)
confusion_log

col_0,0,1
purch,,
0,6752,2
1,174,0


The model is predicting 2 purchases for all customers. However, it did not accurately predict any purchases. The accuracy score is still  not a reliable measure because the target event is still relatively rare.

In [246]:
#Confusion Matrix for Decision Tree
confusion_tree = pd.crosstab(y_test, y_pred_tree)
confusion_tree

col_0,0,1
purch,,
0,6747,7
1,173,1


### Compare Model accuracy

In [160]:
from sklearn.metrics import accuracy_score

print("Accuracy of Logistic Regression:",accuracy_score(y_test, y_preds)),
print("Accuracy of Decision Tree:",accuracy_score(y_test, y_pred_tree))


Accuracy of Logistic Regression: 0.9737086297556449
Accuracy of Decision Tree: 0.9730900092793071


### Compare Model Precision

In [171]:
from sklearn.metrics import average_precision_score

average_precision = average_precision_score(y_test, y_scores)

average_precision_tree = average_precision_score(y_test, y_scores_tree)

print("Accuracy of Logistic Regression:",average_precision.round(3)),
print("Accuracy of Decision Tree:",average_precision_tree.round(3))


Accuracy of Logistic Regression: 0.031
Accuracy of Decision Tree: 0.029


### Compare Recall Scores

In [253]:
#Recall score is the ratio of true positives to all positive values. 
#In other words, of all actual positive values, what percent are accurately
#precicted by the model

from sklearn.metrics import recall_score

print("Recall Score for Logistic Regression:",recall_score(y_test, y_preds)),
print("Recall Score for Decision Tree:",recall_score(y_test, y_pred_tree))

Recall Score for Logistic Regression: 0.0
Recall Score for Decision Tree: 0.005747126436781609


### Compare Misclassification Rates

In [256]:
print("Misclassification for Logistic Regression:",(confusion_log.loc[1,0]+confusion_log.loc[0,1])\
/(confusion_log.sum()[1]+confusion_log.sum()[0])),
print("Misclassification for Decision Tree:",(confusion_tree.loc[1,0]+confusion_tree.loc[0,1])\
/(confusion_tree.sum()[1]+confusion_tree.sum()[0]))

Misclassification for Logistic Regression: 0.025404157043879907
Misclassification for Decision Tree: 0.025981524249422634


In [257]:
#clf.predict_proba(X_test)[:,1].max()

0.9911498646659971